In [1]:
!pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Chunking

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the text file
file_path = "constitution.txt"  # Replace with your actual file path
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()
#print(text)
# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,       # max characters per chunk
    chunk_overlap=70  ,    # overlap between chunks to preserve context
    separators=["\n\n", "\n", "।", ".", " "]
)

# Split the text (pass the variable, not a string)
chunks = text_splitter.split_text(text)
print(f"Created {len(chunks)} chunks ")
# Print the chunks
for i, chunk in enumerate(chunks[:10]):
    print(f"Chunk {i+1}:\n{chunk}\n")


Created 324 chunks 
Chunk 1:
প্রস্তাবনা
আমরা, বাংলাদেশের জনগণ , ১৯৭১ সালের ২৬শে মার্চ স্বাধীনতা ঘোষণা করে জাতীয় মুক্তির ঐতিহাসিক সংগ্রামের মাধ্যমে স্বাধীন ও সার্বভৌম গণপ্রজাতন্ত্রী বাংলাদেশ প্রতিষ্ঠা করেছি।

Chunk 2:
আমরা অঙ্গীকার করছি, সেই সব মহান আদর্শ—যেগুলো আমাদের সাহসী জনগণকে জাতীয় মুক্তি সংগ্রামে আত্মনিয়োগ করতে এবং অনেক বীর শহীদকে জীবন উৎসর্গ করতে অনুপ্রাণিত করেছিল—যেমন: জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা—এই আদর্শগুলোই হবে এই সংবিধানের মূলনীতি।

Chunk 3:
আমরা আরও অঙ্গীকার করছি, আমাদের রাষ্ট্রের অন্যতম লক্ষ্য হবে গণতান্ত্রিক পদ্ধতিতে একটি শোষণমুক্ত সমাজতান্ত্রিক সমাজ গঠন, যেখানে আইনের শাসন, মৌলিক মানবাধিকার এবং রাজনৈতিক, অর্থনৈতিক ও সামাজিক সমতা, স্বাধীনতা ও ন্যায্যতা সব নাগরিকের জন্য নিশ্চিত করা হবে।

Chunk 4:
আমরা জোর দিয়ে ঘোষণা করছি, যেন আমরা স্বাধীন জাতি হিসেবে উন্নতি করতে পারি এবং মানবজাতির অগ্রগতিশীল আশা ও আকাঙ্ক্ষার সঙ্গে সঙ্গতি রেখে আন্তর্জাতিক শান্তি ও সহযোগিতায় সম্পূর্ণ ভূমিকা রাখতে পারি, সে উদ্দেশ্যে এই সংবিধানের শ্রেষ্ঠত্ব বজায় রাখা এবং একে রক্ষা, সমর্থন

# Vector Embedding

In [3]:
from sentence_transformers import SentenceTransformer, LoggingHandler, models, evaluation, losses, util
from torch.utils.data import DataLoader
from sentence_transformers.datasets import ParallelSentencesDataset
import logging
import os
import numpy as np


class BanglaSentenceTransformerSmall:
    def __init__(self, model_name='intfloat/multilingual-e5-large'):
        """Initialize the model once"""
        self.model = SentenceTransformer(model_name)
    def encode(self, sentences):
        """Encode a list of sentences"""
        sentence_embeddings = self.model.encode(sentences)
        return dict(zip(sentences, sentence_embeddings))

In [4]:
from langchain.docstore.document import Document
import chromadb
from langchain.embeddings import SentenceTransformerEmbeddings
import os
os.environ["CHROMA_TELEMETRY_ENABLED"] = "False"
 # Assuming this is your custom class

# 1. Initialize the Bangla Sentence Transformer
model = BanglaSentenceTransformerSmall()

# 2. Encode the chunks
embeddings_dict = model.encode(chunks)

# 3. Prepare documents with the chunk text
documents = [Document(page_content=chunk) for chunk in embeddings_dict.keys()]

# 4. Set up ChromaDB client
client = chromadb.Client()

# 5. Try to get the collection if it already exists
collection_name = "bangla_embeddings"
try:
    collection = client.get_collection(collection_name)
except Exception as e:
    # If collection doesn't exist, create it
    collection = client.create_collection(collection_name)

# 6. Add the sentence embeddings to ChromaDB collection
for idx, (sentence, embedding) in enumerate(embeddings_dict.items()):
    # Ensure idx is defined within the loop, no error in 'for' loop
    collection.add(
        ids=[f"doc_{idx}"],             # Unique ID for each document
        documents=[sentence],           # The sentence content
        metadatas=[{"source": "constitution"}],  # Added a key-value pair for metadata
        embeddings=[embedding]          # The corresponding embedding
    )
#client.persist()

print(f"Stored {len(chunks)} chunks in Chroma vector store.")

#print(f"Stored {len(chunks)} chunks in Chroma vector store.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


Stored 324 chunks in Chroma vector store.


In [5]:
from chromadb.config import Settings
client = chromadb.PersistentClient(path="my_chroma_db")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


# Example of Retrieval

In [6]:
query = "কখন রাষ্ট্রদ্রোহিতা হবে?"
query_embedding = model.encode([query])[query]  # Get the embedding

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3,
    include=["documents", "distances"]  # Important: request distances
)

# Display each result with its similarity score
for doc, score in zip(results['documents'][0], results['distances'][0]):
    print(f"Document: {doc}")
    print(f"Similarity Score: {score:.4f}\n")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Document: ৭ক। সংবিধান বাতিল, স্থগিতকরণ, রাষ্ট্রদ্রোহিতা ইত্যাদি অপরাধ :
১। কেউ যদি শক্তি ব্যবহার করে, বা অন্য কোনো বেআইনি উপায়ে—
এই সংবিধান বা এর কোনো অংশ বাতিল বা স্থগিত করে, বা সেটা করার চেষ্টা করে বা ষড়যন্ত্র করে, বা জনগণের মধ্যে সংবিধানের প্রতি বিশ্বাস কমানোর চেষ্টা করে—
তাহলে সেটি রাষ্ট্রদ্রোহিতা হিসেবে গণ্য হবে। সেই ব্যক্তি রাষ্ট্রদ্রোহের অপরাধে অপরাধী বলে ধরা হবে।
Similarity Score: 0.3422

Document: দ্বিতীয় ভাগ, রাষ্ট্র পরিচালনার মূলনীতি
(৮ )(১) জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা - এই নীতিসমূহ এবং এদের থেকে উদ্ভূত অন্যান্য নীতিসমূহ রাষ্ট্র পরিচালনার মূলনীতি হিসেবে গণ্য হবে।
Similarity Score: 0.3931

Document: ধর্ম, প্রভৃতি কারণে বৈষম্য
২৮।(১) কোনো নাগরিকের প্রতি কেবল ধর্ম, গোষ্ঠী, বর্ণ, লিঙ্গ বা জন্মস্থানের কারণে রাষ্ট্র বৈষম্য করবে না।
(২) রাষ্ট্র ও সমাজের সব স্তরে নারী ও পুরুষের সমান অধিকার নিশ্চিত করবে।
(৩) ধর্ম, গোষ্ঠী, বর্ণ, লিঙ্গ বা জন্মস্থানের কারণে কোনো নাগরিককে বিনোদন বা বিশ্রামের স্থানে প্রবেশ বা শিক্ষা প্রতিষ্ঠানে ভর্তিতে কোনো বাধা, শর্ত বা অযোগ্যতা দেও

# Query Re-writting

In [7]:
import openai
from openai import OpenAI

# Initialize the OpenAI client (replace with your API key)
client = OpenAI(api_key="")
def rewrite_query(query: str, model: str = "gpt-3.5-turbo-0125") -> str:
    """
    বাংলা ভাষায় প্রশ্ন পুনর্লিখন করে যাতে এটি বাংলাদেশের সংবিধানের জন্য RAG-ভিত্তিক QA সিস্টেমের জন্য আরও সুনির্দিষ্ট হয়।

    Args:
        query (str): ব্যবহারকারীর মূল প্রশ্ন বাংলায়।
        model (str): OpenAI মডেলের নাম (ডিফল্ট: gpt-3.5-turbo)।

    Returns:
        str: পুনর্লিখিত প্রশ্ন।
    """
    # Define a prompt in Bangla
    prompt = (
        "আপনি বাংলাদেশের সংবিধানের একজন বিশেষজ্ঞ। নিম্নলিখিত বাংলা প্রশ্নটিকে আরও সুনির্দিষ্ট, পরিষ্কার এবং সংবিধানের প্রাসঙ্গিক অনুচ্ছেদ বা ধারা পুনরুদ্ধারের জন্য উপযুক্ত করে পুনর্লিখন করুন। "
        "পুনর্লিখিত প্রশ্নটি বাংলায় হতে হবে, মূল উদ্দেশ্য বজায় রাখতে হবে এবং সংবিধানের মূল শব্দ বা ধারণার উপর ফোকাস করতে হবে। যদি প্রশ্নের শেষে '?' না থাকে তবে তা যুক্ত করুন,প্রশ্নের অর্থ বজায় রাখুন, অতিরিক্ত কিছু যোগ করবেন না   "
        "মূল প্রশ্ন: {}\nপুনর্লিখিত প্রশ্ন: "
    ).format(query)

    try:
        # Call ChatGPT API
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "আপনি একজন বাংলাদেশের সংবিধান বিশেষজ্ঞ।স্পষ্ট প্রশ্ন তৈরি করুন বাংলায়  ।"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.2
        )

        # Extract the rewritten query
        rewritten_query = response.choices[0].message.content.strip()

        # Remove the prefix if present
        prefix = "পুনর্লিখিত প্রশ্ন: "
        if rewritten_query.startswith(prefix):
            rewritten_query = rewritten_query[len(prefix):].strip()

        return rewritten_query

    except Exception as e:
        return f"Error rewriting query: {str(e)}"

# Example usage
if __name__ == "__main__":
    # Sample Bangla queries
    sample_queries = [
        "বাংলাদেশের সংবিধানে মৌলিক অধিকার কী কী?",
        "সংবিধানের ৭ নং অনুচ্ছেদে কী বলা আছে?",
        "ভোটের অধিকার সম্পর্কে সংবিধানে কী আছে?",
        "আপীল  বিভাগের কাজ  কী  বিস্তারিত বল "
    ]

    for query in sample_queries:
        rewritten = rewrite_query(query, model="gpt-3.5-turbo-0125")
        print(f"মূল প্রশ্ন: {query}")
        print(f"পুনর্লিখিত প্রশ্ন: {rewritten}\n")


মূল প্রশ্ন: বাংলাদেশের সংবিধানে মৌলিক অধিকার কী কী?
পুনর্লিখিত প্রশ্ন: বাংলাদেশের সংবিধানে মৌলিক অধিকার কী কী উল্লেখিত?

মূল প্রশ্ন: সংবিধানের ৭ নং অনুচ্ছেদে কী বলা আছে?
পুনর্লিখিত প্রশ্ন: সংবিধানের ৭ নং অনুচ্ছেদে কী বলা আছে?

মূল প্রশ্ন: ভোটের অধিকার সম্পর্কে সংবিধানে কী আছে?
পুনর্লিখিত প্রশ্ন: ভোটের অধিকার সম্পর্কে সংবিধানে কী উল্লেখিত আছে?

মূল প্রশ্ন: আপীল  বিভাগের কাজ  কী  বিস্তারিত বল 
পুনর্লিখিত প্রশ্ন: আপীল বিভাগের কাজ কী? সংবিধানের কোন ধারা আপীল বিভাগের কাজ সম্পর্কে বিস্তারিত তথ্য প্রদান করে?



# Query Classification

In [8]:

import openai
from openai import OpenAI

# Initialize the OpenAI client (replace with your API key)
#client = OpenAI(api_key="YOUR_OPENAI_API_KEY")

def classify_question(question: str, model: str = "gpt-3.5-turbo-0125") -> dict:
    """
    বাংলা ভাষায় একটি প্রশ্নকে ফ্যাক্টয়েড বা বর্ণনামূলক হিসেবে শ্রেণীবদ্ধ করে, বাংলাদেশের সংবিধানের প্রেক্ষাপটে।

    Args:
        question (str): ব্যবহারকারীর প্রশ্ন বাংলায়।
        model (str): OpenAI মডেলের নাম (ডিফল্ট: gpt-3.5-turbo)।

    Returns:
        dict: শ্রেণীবদ্ধকরণ ('ফ্যাক্টয়েড' বা 'বর্ণনামূলক') ।
    """
    # Define a prompt in Bangla
    prompt = (
        "আপনি বাংলাদেশের সংবিধানের একজন বিশেষজ্ঞ। নিম্নলিখিত বাংলা প্রশ্নটিকে 'ফ্যাক্টয়েড' বা 'বর্ণনামূলক' হিসেবে শ্রেণীবদ্ধ করুন। "
        "ফ্যাক্টয়েড প্রশ্নগুলো সুনির্দিষ্ট,  সংক্ষিপ্ত, সরাসরি উত্তরযোগ্য । "
        "বর্ণনামূলক প্রশ্নগুলো ব্যাখ্যা বা বিশ্লেষণের প্রয়োজন, অথবা সংশোধনী বা ঐতিহাসিক প্রেক্ষাপট জড়িত। "
        "যদি প্রশ্নের শেষে '?' না থাকে, তবে '?' যুক্ত করুন । "
        "শ্রেণীবদ্ধকরণ সঠিক হতে হবে  "
        "উত্তরে শুধুমাত্র শ্রেণীবদ্ধকরণ ('ফ্যাক্টয়েড' বা 'বর্ণনামূলক') । "
        "উত্তর ফরম্যাট: \nশ্রেণীবদ্ধকরণ: [ফ্যাক্টয়েড/বর্ণনামূলক]\n"
        "প্রশ্ন: {}"
    ).format(question)

    try:
        # Call ChatGPT API
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "আপনি একজন বাংলাদেশের সংবিধান বিশেষজ্ঞ। সব উত্তর বাংলায়, সঠিক এবং সম্পূর্ণ বাক্যে হতে হবে।যদি প্রশ্নের শেষে '?' না থাকে, তবে '?' যুক্ত করুন । "},
                {"role": "user", "content": prompt}
            ],
            max_tokens=250,  # Increased to ensure complete explanations
            temperature=0.2  # Lower for consistent, accurate classifications
        )

        # Extract the classification and explanation
        response_text = response.choices[0].message.content.strip()

        # Parse the response
        classification = "বর্ণনামূলক"  # Default to descriptive if parsing fails
        explanation = "পার্সিং ত্রুটি: উত্তর থেকে শ্রেণীবদ্ধকরণ পাওয়া যায়নি।"
        lines = response_text.split("\n")
        for line in lines:
            if line.startswith("শ্রেণীবদ্ধকরণ:"):
                classification = line.replace("শ্রেণীবদ্ধকরণ:", "").strip()


        # Validate classification
        if classification not in ["ফ্যাক্টয়েড", "বর্ণনামূলক"]:
            classification = "বর্ণনামূলক"
            explanation = "ত্রুটি: অবৈধ শ্রেণীবদ্ধকরণ প্রাপ্ত। বর্ণনামূলক হিসেবে ডিফল্ট করা হয়েছে।"

        return {"classification": classification, "explanation": explanation}

    except Exception as e:
        return {"classification": "বর্ণনামূলক", "explanation": f"ত্রুটি: {str(e)}"}

# Example usage
if __name__ == "__main__":
    # Sample Bangla questions
    sample_questions = [
        "বাংলাদেশের সংবিধানে মৌলিক অধিকার কী কী?",
        "সংবিধানের ৭ নং অনুচ্ছেদে কী বলা আছে?",
        "ভোটের অধিকার সম্পর্কে সংবিধানে কী আছে?",
        "সংবিধানে অধিকার কী?",
        "১৫তম সংশোধনীতে ভোটের অধিকারে কী পরিবর্তন হয়েছে?",
        "বাংলাদেশের সংবিধানে অধিকার কী?",
        "সংবিধানে নাগরিকদের স্বাধীনতা কীভাবে সুরক্ষিত হয়?"
    ]

    for question in sample_questions:
        result = classify_question(question, model="gpt-3.5-turbo-0125")
        print(f"প্রশ্ন: {question}")
        print(f"শ্রেণীবদ্ধকরণ: {result['classification']}")
        #print(f"ব্যাখ্যা: {result['explanation']}\n")

প্রশ্ন: বাংলাদেশের সংবিধানে মৌলিক অধিকার কী কী?
শ্রেণীবদ্ধকরণ: ফ্যাক্টয়েড
প্রশ্ন: সংবিধানের ৭ নং অনুচ্ছেদে কী বলা আছে?
শ্রেণীবদ্ধকরণ: ফ্যাক্টয়েড
প্রশ্ন: ভোটের অধিকার সম্পর্কে সংবিধানে কী আছে?
শ্রেণীবদ্ধকরণ: ফ্যাক্টয়েড
প্রশ্ন: সংবিধানে অধিকার কী?
শ্রেণীবদ্ধকরণ: ফ্যাক্টয়েড
প্রশ্ন: ১৫তম সংশোধনীতে ভোটের অধিকারে কী পরিবর্তন হয়েছে?
শ্রেণীবদ্ধকরণ: ফ্যাক্টয়েড
প্রশ্ন: বাংলাদেশের সংবিধানে অধিকার কী?
শ্রেণীবদ্ধকরণ: ফ্যাক্টয়েড
প্রশ্ন: সংবিধানে নাগরিকদের স্বাধীনতা কীভাবে সুরক্ষিত হয়?
শ্রেণীবদ্ধকরণ: বর্ণনামূলক


In [15]:
question="আপীল বিভাগের কাজ কী বিস্তারিত বল"
result = classify_question(question)
print(f"প্রশ্ন: {question}")
print(f"শ্রেণীবদ্ধকরণ: {result['classification']}")

প্রশ্ন: আপীল বিভাগের কাজ কী বিস্তারিত বল
শ্রেণীবদ্ধকরণ: বর্ণনামূলক


# Generation Part

In [9]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain.memory import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import SentenceTransformer

In [10]:
# Step 1: Initialize models
#embedding_model = SentenceTransformer('sagorsarker/bangla-bert-base')  # Better for Bengali
import os
os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(
    openai_api_key="",
    model_name="gpt-3.5-turbo-0125",  # Use a model with better multilingual support
    temperature=0
)
parser = StrOutputParser()

# Chat History

In [11]:
contexts=[]
def add_context(item):
    global contexts  # Tell Python to use the global 'contexts'
    contexts.append(item)


In [12]:
# Step 2: Initialize Chat History
chat_history = ChatMessageHistory()
def format_chat_history(history, max_messages=10):
    """Convert chat history into a formatted string."""
    messages = history.messages[-max_messages:]
    formatted = ""
    for msg in messages:
        if isinstance(msg, HumanMessage):
            formatted += f"User: {msg.content}\n"
        elif isinstance(msg, AIMessage):
            formatted += f"Assistant: {msg.content}\n"
            #print(msg.content)
            #print("-----------------")
            add_context(msg.content)
    return formatted

# Format Context

In [13]:
# Step 3: Define a function to format documents
def format_docs(results):
    """Format vector store results into a string."""
    #print("Raw results structure:", results)  # Debug full results
    documents = results.get('documents', [[]])[0]  # Get first list of documents
    #print("Extracted documents:", documents)  # Debug extracted documents

    if not documents:
        print("Warning: No documents retrieved!")
        return ""
    formatted = "\n".join([str(doc) for doc in documents if doc])  # Ensure no empty strings
    for doc in documents:
      add_context(str(doc))
    #print("Formatted context:", formatted[:200], "...")  # Debug formatted output
    #if not formatted.strip():
        #print("Warning: Formatted context is empty after processing!")
    #formatted=filtered_contexts(results)
    #print(formatted)
    return formatted  # Limit context length

In [14]:
# Step 4: Define the vector store query function
def query_vector_store(question):
    embedding = model.encode([question])[question]
    results = collection.query(query_embeddings=[embedding], n_results=3)
    return results

In [15]:
# Step 5: Wrap components in LangChain Runnables
query_chain = RunnableLambda(query_vector_store)
context_chain = query_chain | RunnableLambda(format_docs)

# Prompt

In [16]:
# Step 6: Update the Prompt Template
prompt = PromptTemplate(
    template="""
      আপনি বাংলাদেশের সংবিধানের একটি সহায়ক সহকারী।
      শুধুমাত্র নিচের relevant context অংশের ভিত্তিতে প্রশ্নের সঠিক এবং সংক্ষিপ্ত উত্তর দিন।
      চ্যাট ইতিহাস:
      {history}
      প্রাসঙ্গিক অংশ:
      {context}
      প্রশ্ন:
      {question}
      উত্তর শুধুমাত্র প্রাসঙ্গিক অংশ থেকে নিন। Context এর  বাইরের কোনো তথ্য, অনুমান, বা সাধারণ জ্ঞান ব্যবহার করবেন না।
      যদি উত্তর প্রাসঙ্গিক অংশে না থাকে, তবে বলুন: "আমি জানি না"
      যদি উত্তর প্রাসঙ্গিক অংশে  থাকে, তবে  উত্তর দিন বাংলায়, বিস্তারিত  এবং সঠিক ।
    """,
    input_variables=['history', 'context', 'question']
)

In [17]:
# Step 7: Update the parallel chain to include history
def get_history(_):
    return format_chat_history(chat_history)

parallel_chain = RunnableParallel({
    'history': RunnableLambda(get_history),
    'context': context_chain,
    'question': RunnablePassthrough()
})

In [18]:

# Step 8: Define the main chain with debugging
def debug_invoke(input):
    #print("Prompt input:", input)
    return prompt.invoke(input)

main_chain = parallel_chain | debug_invoke | llm | parser

# Ask A Question

In [44]:
# Step 9: Function to invoke the chain and update history
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
def ask_question(question):
    try:
        contexts.clear();
        result = main_chain.invoke(question)
        #print(contexts)
        if not contexts:
          return "আমি জানি না"
        #print(contexts)
        answer_similarity=0
        for context in contexts:
          true_answer_embedding = model.encode([result])[result]
          generated_answer_embedding = model.encode([context])[context]
          answer_similarity1 = cosine_similarity([true_answer_embedding], [generated_answer_embedding])[0][0]
          if answer_similarity1>answer_similarity:
            answer_similarity=answer_similarity1
          #print(context)
          #print(answer_similarity1)
            #break
          if(answer_similarity>=0.9):
            break

        if answer_similarity<0.8:
          return "আমি জানি না"
        #print("Raw LLM output:", result)
        chat_history.add_message(HumanMessage(content=question))
        chat_history.add_message(AIMessage(content=result))
        return result
    except Exception as e:
        print(f"Error during generation: {e}")
        return None

In [47]:
def final_ans(query):

  result = classify_question(query)
  #print(f"শ্রেণীবদ্ধকরণ: {result['classification']}")

  if result['classification'] == "বর্ণনামূলক":
      rewritten = rewrite_query(query)
      #print(f"মূল প্রশ্ন: {query}")
      #p#rint(f"পুনর্লিখিত প্রশ্ন: {rewritten}\n")

      #rewritten_queries = generate_multi_queries(rewritten)
      #print(f"মূল প্রশ্ন: {query}")
      #for i, rewritten in enumerate(rewritten_queries, 1):
          #print(f"পুনর্লিখিত প্রশ্ন {i}: {rewritten}")
      ans=ask_question(rewritten)
      return ans
  else:
    ans=ask_question(query)
    return ans


In [48]:
query = "আপীল  বিভাগের কাজ  কী  বিস্তারিত বল ?"
print(final_ans(query))

আপীল বিভাগের কাজ হলো হাইকোর্ট বিভাগের রায়, ডিক্রী, আদেশ বা দণ্ডাদেশের বিরুদ্ধে আপীল শুনানী এবং তা নিষ্পত্তির এখতিয়ার আপীল বিভাগের থাকবে। হাইকোর্ট বিভাগের রায়, ডিক্রী, আদেশ বা দণ্ডাদেশের বিরুদ্ধে আপীল বিভাগের নিকট সেই ক্ষেত্রে অধিকারবলে আপীল করা যাবে।


In [49]:
query = "আইন কীভাবে প্রণয়ন করা হয়?"
print(final_ans(query))

আইন প্রণয়নের জন্য সংসদে যে কোনো প্রস্তাব বিল আকারে উত্থাপিত হবে এবং সংসদ কোনো বিল গৃহীত করলে তা রাষ্ট্রপতির কাছে সম্মতির জন্য পাঠাতে হবে। এছাড়া, সংসদ কর্তৃক প্রণীত যে কোনো আইন-সাপেক্ষে, সুপ্রীম কোর্ট রাষ্ট্রপতির অনুমোদন নিয়ে প্রত্যেক বিভাগের এবং অধঃস্তন যে কোনো আদালতের রীতি ও পদ্ধতি-নিয়ন্ত্রণের জন্য বিধিসমূহ প্রণয়ন করতে পারবে।


In [50]:
query = "কখন স্বাধীনতার ঘোষণা দেওয়া হয়?"
print(final_ans(query))

স্বাধীনতার ঘোষণা ১৯৭১ সালের ২৬শে মার্চে দেওয়া হয়।


In [51]:
query = "সংবিধানে মানবাধিকার নিয়ে কী বলা আছে?"
print(final_ans(query))

উত্তর: সংবিধানে মানবাধিকার নিয়ে বলা আছে যে, কোনো ব্যক্তিকে আইন অনুযায়ী ছাড়া তার জীবন বা ব্যক্তিস্বাধীনতা থেকে বঞ্চিত করা যাবে না। এটি মানবাধিকার সংরক্ষণের একটি গুরুত্বপূর্ণ সূত্র।


In [52]:
query = "কখন রাষ্ট্রদ্রোহিতা হবে?"
print(final_ans(query))

যদি কেউ শক্তি ব্যবহার করে, বা অন্য কোনো বেআইনি উপায়ে সংবিধান বা এর কোনো অংশ বাতিল বা স্থগিত করে, বা সেটা করার চেষ্টা করে বা ষড়যন্ত্র করে, বা জনগণের মধ্যে সংবিধানের প্রতি বিশ্বাস কমানোর চেষ্টা করে তাহলে সেটি রাষ্ট্রদ্রোহিতা হিসেবে গণ্য হবে এবং সেই ব্যক্তি রাষ্ট্রদ্রোহের অপরাধে অপরাধী হিসেবে ধরা হবে।


In [55]:
query = " জাতীয় প্রতীক কেমন ?"
print(final_ans(query))

জাতীয় প্রতীকের বর্ণনা অনুযায়ী, পানির উপর ভাসছে জাতীয় ফুল শাপলা। চারপাশে ঘিরে আছে ধানের শীষ। উপরে আছে তিনটি পাটপাতা এবং দুই পাশে দুটি তারা (তারকা)।


In [56]:
query = "এটির  চারপাশে কী রয়েছে?"
print(final_ans(query))

চারপাশে জাতীয় প্রতীকে ঘিরে আছে ধানের শীষ, তিনটি পাটপাতা এবং দুই পাশে দুটি তারা (তারকা)।


In [57]:

query = "বাংলাদেশের প্রধানমন্ত্রী কে?"
final_ans(query)

'আমি জানি না'

In [58]:

query = "কোন ব্যাক্তি গ্রেফতার হলে তার অধিকারসমূহ কী কী বিস্তারিত বল?"
print(final_ans(query))

গ্রেপ্তার হলে ব্যক্তিকে যত দ্রুত সম্ভব গ্রেপ্তার হওয়ার কারণ জানানো হবে এবং তাকে নির্দিষ্ট করা সময়ের বেশি প্রহরায় আটক রাখা যাবে না। গ্রেপ্তার ব্যক্তির আইনজীবীর সঙ্গে পরামর্শ করার এবং আত্মপক্ষ সমর্থনের অধিকার থেকে বঞ্চিত করা যাবে না। গ্রেপ্তারকৃত বা প্রহরায় আটক ব্যক্তিকে গ্রেপ্তারের ২৪ ঘণ্টার মধ্যে নিকটম ম্যাজিস্ট্রেটের সামনে হাজির করতে হবে। ম্যাজিস্ট্রেটের অনুমতি ছাড়া তাকে অতিরিক্ত সময় আটক রাখা যাবে না। এই বিধানগুলি প্রযোজ্য হবে না যদি ব্যক্তি বর্তমানে বিদেশী শত্রু হয় অথবা যাঁকে নিবর্তনমূলক আটকের আইন অনুসারে গ্রেপ্তার বা আটক করা হয়েছে।


In [59]:

query = "কোন ব্যাক্তি গ্রেফতার হলে তার সাথে করণীয় কী কী ?"
print(final_ans(query))

গ্রেপ্তার হলে ব্যক্তিকে যত দ্রুত সম্ভব গ্রেপ্তার হওয়ার কারণ জানানো হবে এবং তাকে নির্দিষ্ট করা সময়ের বেশি প্রহরায় আটক রাখা যাবে না। গ্রেপ্তার ব্যক্তির আইনজীবীর সঙ্গে পরামর্শ করার এবং আত্মপক্ষ সমর্থনের অধিকার থেকে বঞ্চিত করা যাবে না। গ্রেপ্তারকৃত বা প্রহরায় আটক ব্যক্তিকে গ্রেপ্তারের ২৪ ঘণ্টার মধ্যে নিকটতম ম্যাজিস্ট্রেটের সামনে হাজির করতে হবে। ম্যাজিস্ট্রেটের অনুমতি ছাড়া তাকে অতিরিক্ত সময় আটক রাখা যাবে না। এই বিধানগুলি প্রযোজ্য হবে না যদি ব্যক্তি বর্তমানে বিদেশী শত্রু হয় অথবা যাঁকে নিবর্তনমূলক আটকের আইন অনুসারে গ্রেপ্তার বা আটক করা হয়েছে।


In [60]:
query = "তাকে কার সামনে হাজির করতে হবে ?"
print(final_ans(query))

গ্রেপ্তারকৃত বা প্রহরায় আটক ব্যক্তিকে গ্রেপ্তারের ২৪ ঘণ্টার মধ্যে নিকটম ম্যাজিস্ট্রেটের সামনে হাজির করতে হবে।


In [61]:
query = "তাকে কত সময়য়ের মধ্যে ম্যাজিস্ট্রেটের সামনে হাজির করতে হবে? "
final_ans(query)

'গ্রেপ্তারকৃত বা প্রহরায় আটক ব্যক্তিকে গ্রেপ্তারের ২৪ ঘণ্টার মধ্যে নিকটতম ম্যাজিস্ট্রেটের সামনে হাজির করতে হবে।'

In [62]:
# Example usage
query = "নির্বাচন কমিশনের দায়িত্বসমূহ কী কী ?  "
print(final_ans(query))

প্রশ্নটির উত্তর: নির্বাচন কমিশনের দায়িত্বসমূহ হলো রাষ্ট্রপতি ও সংসদের নির্বাচনের জন্য ভোটার-তালিকা প্রস্তুতকরণের তত্ত্বাবধান, নির্দেশনা ও নিয়ন্ত্রণ এবং সংশ্লিষ্ট নির্বাচন পরিচালনার দায়িত্ব নির্বাচন কমিশনের উপর থাকবে। এছাড়া তারা রাষ্ট্রপতির পদ এবং সংসদের নির্বাচনের জন্য ভোটার-তালিকা প্রস্তুত করবেন।


In [63]:
# Example usage
query = "কখন জরুরী অবস্থা জারি করা হয় ? "
final_ans(query)

'জরুরী অবস্থা জারি করা হয় যখন রাষ্ট্রপতি সন্তোষজনকভাবে প্রমাণিত করেন যে বাংলাদেশ বা এর কোনো অংশে যুদ্ধ, বহিরাগত আক্রমণ বা অভ্যন্তরীণ বিশৃঙ্খলার কারণে নিরাপত্তা বা অর্থনৈতিক জীবন বিপন্ন হয়েছে। তারপর তিনি সর্বোচ্চ ১২০ দিনের জন্য জরুরী অবস্থা ঘোষণা করতে পারবেন।'

In [64]:
# Example usage
query = "জাতীয় সঙ্গীত ? "
print(final_ans(query))

আমার সোনার বাংলা হল বাংলাদেশের জাতীয় সঙ্গীত।


In [39]:
import pandas as pd

# Data
data = {
    "Question": [

        "দেশের সর্বোচ্চ আইন কী?",
        "বাংলাদেশের জাতীয় সঙ্গীত কী?",
        "রাষ্ট্রের মূলনীতিসমূহ কী কী?",
        "সংবিধানের সংশোধন কিভাবে হয়?"
    ],
    "Context": [

        "বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।",
        "বাংলাদেশের জাতীয় সঙ্গীত 'আমার সোনার বাংলা'।",
        "রাষ্ট্রের মূলনীতি হিসেবে গণতন্ত্র, সমাজবাদী রাষ্ট্রব্যবস্থা, এবং ধর্মনিরপেক্ষতা বলা হয়েছে।",
        "সংবিধানের সংশোধন করতে সংসদে বিশেষ প্রস্তাব পাস করতে হয়।"
    ],
    "Answer": [

        "বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।",
        "'আমার সোনার বাংলা' জাতীয় সঙ্গীত।",
        "গণতন্ত্র, সমাজবাদী রাষ্ট্রব্যবস্থা, ধর্মনিরপেক্ষতা।",
        "সংসদে বিশেষ প্রস্তাব পাসের মাধ্যমে সংশোধন হয়।"
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('bangla_constitution_part1_question_ans.csv', index=False)

print("CSV file with questions, context, and answers has been created successfully.")


CSV file with questions, context, and answers has been created successfully.


In [40]:
# prompt: load the csv file

import pandas as pd

# Assuming the CSV file 'bangla_constitution_qa.csv' is in the current directory
try:
    df = pd.read_csv('/content/bangla_constitution_part1_question_ans.csv')
    print("CSV file loaded successfully.")
    # Now you can work with the DataFrame 'df'
    #print(df.head())  # Print the first few rows to verify

except FileNotFoundError:
    print("Error: 'bangla_constitution_part1_question_ans.csv.csv' not found in the current directory.")
except pd.errors.EmptyDataError:
    print("Error: 'bangla_constitution_part1_question_ans.csv' is empty.")
except pd.errors.ParserError:
    print("Error: Unable to parse the CSV file. Check its format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


CSV file loaded successfully.


In [65]:
from sklearn.metrics.pairwise import cosine_similarity

#modell = SentenceTransformer('shihab17/bangla-sentence-transformer')
def evaluate_retrieval_accuracy(dataset, model, k=5, similarity_threshold=0.7):
    retrieval_hits = 0
    total_questions = len(dataset)

    for _, example in dataset.iterrows():  # Iterate over rows of the DataFrame
        question = example['Question']
        context = example['Context']
        true_answer = example['Answer']

        generated_ans=final_ans(question);

        print(question)
        print('question: ', question)
        print("answer ",true_answer)
        print("gen_ans ",generated_ans)


        print("--------------------------------------------------------------------------------------\n\n")


# Example usage with your DataFrame (df):
retrieval_accuracy = evaluate_retrieval_accuracy(df, model)
#print(f"Retrieval Accuracy: {retrieval_accuracy:.2f}")


দেশের সর্বোচ্চ আইন কী?
question:  দেশের সর্বোচ্চ আইন কী?
answer  বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।
gen_ans  সংবিধান হলো দেশের সর্বোচ্চ আইন। যদি কোনো আইন সংবিধানের বিরোধিতা করে, তাহলে সেই আইনের অসংগত অংশ বাতিল বলে গণ্য হবে।
--------------------------------------------------------------------------------------


বাংলাদেশের জাতীয় সঙ্গীত কী?
question:  বাংলাদেশের জাতীয় সঙ্গীত কী?
answer  'আমার সোনার বাংলা' জাতীয় সঙ্গীত।
gen_ans  আমার সোনার বাংলা হল বাংলাদেশের জাতীয় সঙ্গীত।
--------------------------------------------------------------------------------------


রাষ্ট্রের মূলনীতিসমূহ কী কী?
question:  রাষ্ট্রের মূলনীতিসমূহ কী কী?
answer  গণতন্ত্র, সমাজবাদী রাষ্ট্রব্যবস্থা, ধর্মনিরপেক্ষতা।
gen_ans  উত্তর: রাষ্ট্রের মূলনীতিসমূহ হলো জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা। এই নীতিসমূহ বাংলাদেশের পরিচালনার মূল সূত্র হবে এবং এই আদর্শগুলোই হবে সংবিধানের মূলনীতি। আইন তৈরির সময় রাষ্ট্র এগুলো মানবে, আইন ও সংবিধান ব্যাখ্যা করার সময় এগুলো নির্দেশক হবে, আর রাষ্ট্র ও নাগরিকদের কাজের 

In [42]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import os

class BanglaSentenceTransformerSmall:
    def __init__(self, model_name='intfloat/multilingual-e5-large'):
        self.model = SentenceTransformer(model_name)

    def encode(self, sentences):
        sentence_embeddings = self.model.encode(sentences)
        return dict(zip(sentences, sentence_embeddings))

    def similarity(self, embedding1, embedding2):
        return util.pytorch_cos_sim(embedding1, embedding2)

def get_gpt_similarity(question, true_answer, generated_answer, llm):
    # Prompt for GPT to score semantic similarity
    prompt = f"""
    Given the question, true answer, and generated answer, score how semantically similar the generated answer is to the true answer on a scale of 0 to 1, where 1 is identical in meaning and 0 is completely different.

    Question: {question}
    True Answer: {true_answer}
    Generated Answer: {generated_answer}

    Provide a score (0–1)  in the format:
    Score: <score>
    """

    # Use ChatOpenAI's invoke method
    messages = [
        SystemMessage(content="You are an evaluator of semantic similarity."),
        HumanMessage(content=prompt)
    ]
    try:
        response = llm.invoke(messages)
        response_text = response.content

        # Parse score from response
        score_line = [line for line in response_text.split('\n') if line.startswith("Score: ")]
        if score_line:
            score = float(score_line[0].replace("Score: ", ""))
            return score
        else:
            print("Warning: Could not parse GPT score. Returning 0.")
            return 0.0
    except Exception as e:
        print(f"Error calling GPT: {e}")
        return 0.0

def evaluate_retrieval_accuracy(dataset, model, llm, k=1, similarity_threshold=0.7):
    y_true = []
    y_pred = []
    recall_scores, precision_scores, f1_scores, llm_reteval_cos_scores, llm_reteval_gpt_scores = [], [], [], [], []
    total_questions = len(dataset)

    for _, example in dataset.iterrows():
        question = example['Question']
        true_answer = example['Answer']

        # Placeholder for main_chain (replace with main_chain.invoke(question))
        generated_ans = f"Generated answer for: {main_chain.invoke(question)}"

        y_true.append(true_answer)
        y_pred.append(generated_ans)

        # Compute cosine similarity
        true_answer_embedding = model.encode([true_answer])[true_answer]
        generated_answer_embedding = model.encode([generated_ans])[generated_ans]
        answer_similarity = cosine_similarity([true_answer_embedding], [generated_answer_embedding])[0][0]

        # Determine relevance (cosine similarity >= threshold)
        is_relevant = 1 if answer_similarity >= similarity_threshold else 0
        total_relevant = 1  # One true answer per question

        # Recall@1
        recall = is_relevant / total_relevant
        recall_scores.append(recall)

        # Precision@1
        precision = is_relevant / k
        precision_scores.append(precision)

        # F1@1
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)

        # LLM-retEval (Cosine)
        llm_reteval_cos_scores.append(answer_similarity)

        # LLM-retEval (GPT)
        llm_reteval_gpt = get_gpt_similarity(question, true_answer, generated_ans, llm)
        llm_reteval_gpt_scores.append(llm_reteval_gpt)

        # Print results
        print(f"question: {question}")
        print(f"answer: {true_answer}")
        print(f"gen_ans: {generated_ans}")
        print(f"Answer Cosine Similarity: {answer_similarity:.3f}")
        print(f"Relevant (Cosine >= {similarity_threshold}): {is_relevant}")
        print(f"Recall@{k}: {recall:.3f}")
        print(f"Precision@{k}: {precision:.3f}")
        print(f"F1@{k}: {f1:.3f}")
        print(f"LLM-retEval (Cosine): {answer_similarity:.3f}")
        print(f"LLM-retEval (GPT): {llm_reteval_gpt:.3f}")
        print("--------------------------------------------------------------------------------------\n")

    # Aggregate results
    avg_recall = np.mean(recall_scores)
    avg_precision = np.mean(precision_scores)
    avg_f1 = np.mean(f1_scores)
    avg_llm_reteval_cos = np.mean(llm_reteval_cos_scores)
    avg_llm_reteval_gpt = np.mean(llm_reteval_gpt_scores)

    return {
        f"Recall@{k}": avg_recall,
        f"Precision@{k}": avg_precision,
        f"F1@{k}": avg_f1,
        "LLM-retEval (Cosine)": avg_llm_reteval_cos,
        "LLM-retEval (GPT)": avg_llm_reteval_gpt,
        "y_true": y_true,
        "y_pred": y_pred
    }


# Initialize models
model = BanglaSentenceTransformerSmall()
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", api_key=os.getenv("OPENAI_API_KEY"))

# Calculate metrics
k = 1
results = evaluate_retrieval_accuracy(df, model, llm, k=k, similarity_threshold=0.7)

# Print aggregated results
print("\nAggregated Results:")
for metric, value in results.items():
    if metric not in ["y_true", "y_pred"]:
        print(f"{metric}: {value:.3f}")
    else:
        print(f"{metric}: {value}")

question: দেশের সর্বোচ্চ আইন কী?
answer: বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।
gen_ans: Generated answer for: সংবিধান।
Answer Cosine Similarity: 0.837
Relevant (Cosine >= 0.7): 1
Recall@1: 1.000
Precision@1: 1.000
F1@1: 1.000
LLM-retEval (Cosine): 0.837
LLM-retEval (GPT): 0.500
--------------------------------------------------------------------------------------

question: বাংলাদেশের জাতীয় সঙ্গীত কী?
answer: 'আমার সোনার বাংলা' জাতীয় সঙ্গীত।
gen_ans: Generated answer for: আমার সোনার বাংলা" গানটির প্রথম ১০টি লাইন।
Answer Cosine Similarity: 0.868
Relevant (Cosine >= 0.7): 1
Recall@1: 1.000
Precision@1: 1.000
F1@1: 1.000
LLM-retEval (Cosine): 0.868
LLM-retEval (GPT): 0.200
--------------------------------------------------------------------------------------

question: রাষ্ট্রের মূলনীতিসমূহ কী কী?
answer: গণতন্ত্র, সমাজবাদী রাষ্ট্রব্যবস্থা, ধর্মনিরপেক্ষতা।
gen_ans: Generated answer for: উত্তর: রাষ্ট্রের মূলনীতিসমূহ হলো জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা। এই নীতিসমূহ এবং 

In [45]:
!pip install ragas

In [46]:
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness

# Load your CSV dataset
df = pd.read_csv('bangla_constitution_part1_question_ans.csv')

questions = df['Question'].tolist()
ground_truth_answers = df['Answer'].tolist()
retrieved_contexts = df['Context'].apply(lambda x: [x]).tolist()

# Generate answers by calling your LLM for each question
generated_answers = []
for q in questions:
    # Replace with your actual LLM call, e.g. main_chain.invoke(q)
    answer = final_ans(q)  # Make sure main_chain.invoke returns a string answer
    generated_answers.append(answer)

# Prepare dataset for evaluation
data_samples = {
    'question': questions,
    'answer': generated_answers,
    'contexts': retrieved_contexts,
    'ground_truth': ground_truth_answers
}
dataset = Dataset.from_dict(data_samples)

# Evaluate with RAGAS metrics
score = evaluate(dataset, metrics=[faithfulness, answer_correctness])

# Convert scores to pandas DataFrame for per-question scores
df_scores = score.to_pandas()

# Print scores question-wise
for i, question in enumerate(questions):
    correctness = df_scores.loc[i, 'answer_correctness']
    faithful = df_scores.loc[i, 'faithfulness']
    print(f"Question {i+1}: {question}")
    print(f"Answer {i+1}: {ground_truth_answers[i]}")
    print(f"Generated Answer: {generated_answers[i]}")
    print(f"Answer Correctness Score: {correctness:.3f}")
    print(f"Faithfulness Score: {faithful:.3f}")
    print('-' * 50)


Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

Question 1: দেশের সর্বোচ্চ আইন কী?
Answer 1: বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।
Generated Answer: সংবিধান হলো দেশের সর্বোচ্চ আইন। যদি কোনো আইন সংবিধানের বিরোধিতা করে, তাহলে সেই আইনের অসংগত অংশ বাতিল বলে গণ্য হবে।
Answer Correctness Score: 0.607
Faithfulness Score: 0.500
--------------------------------------------------
Question 2: বাংলাদেশের জাতীয় সঙ্গীত কী?
Answer 2: 'আমার সোনার বাংলা' জাতীয় সঙ্গীত।
Generated Answer: আমার সোনার বাংলা" গানটির প্রথম ১০টি লাইন।
Answer Correctness Score: 0.728
Faithfulness Score: 0.500
--------------------------------------------------
Question 3: রাষ্ট্রের মূলনীতিসমূহ কী কী?
Answer 3: গণতন্ত্র, সমাজবাদী রাষ্ট্রব্যবস্থা, ধর্মনিরপেক্ষতা।
Generated Answer: উত্তর: রাষ্ট্রের মূলনীতিসমূহ হলো জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা। এই নীতিসমূহ এবং এদের থেকে উদ্ভূত অন্যান্য নীতিসমূহ রাষ্ট্র পরিচালনার মূলনীতি হিসেবে গণ্য হবে। সংবিধানে এই মূলনীতিসমূহ নির্ধারণ করা হয়েছে যেগুলো রাষ্ট্রের মূলনীতি ও মৌলিক মূল্যাবান ধারণাগুলি নির্ধারণ করে।
Answer Corr